In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/travel-insurance-prediction-data/TravelInsurancePrediction.csv


In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(context='talk', style='darkgrid', font='MS GOTHIC')

In [4]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [5]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [64]:
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold

In [7]:
# df = pd.read_csv('/kaggle/input/travel-insurance-prediction-data/TravelInsurancePrediction.csv', index_col=0)
df = pd.read_csv('/kaggle/input/travel-insurance-prediction-data/TravelInsurancePrediction.csv')
df.drop('Unnamed: 0', axis=1, inplace=True) # 1行目のindex_colと同じ処理
df.head()

,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,31,Government Sector,Yes,400000,6,1,No,No,0
1,31,Private Sector/Self Employed,Yes,1250000,7,0,No,No,0
2,34,Private Sector/Self Employed,Yes,500000,4,1,No,No,1
3,28,Private Sector/Self Employed,Yes,700000,3,1,No,No,0
4,28,Private Sector/Self Employed,Yes,700000,8,1,Yes,No,0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1987 entries, 0 to 1986
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Age                  1987 non-null   int64 
 1   Employment Type      1987 non-null   object
 2   GraduateOrNot        1987 non-null   object
 3   AnnualIncome         1987 non-null   int64 
 4   FamilyMembers        1987 non-null   int64 
 5   ChronicDiseases      1987 non-null   int64 
 6   FrequentFlyer        1987 non-null   object
 7   EverTravelledAbroad  1987 non-null   object
 8   TravelInsurance      1987 non-null   int64 
dtypes: int64(5), object(4)
memory usage: 139.8+ KB


In [9]:
for col in df.columns:
    print(f'uniques of {col} is ( {df[col].nunique()} ) : \n {df[col].unique()}')
    print('='*20)

uniques of Age is ( 11 ) : 
 [31 34 28 25 33 26 32 29 35 30 27]
uniques of Employment Type is ( 2 ) : 
 ['Government Sector' 'Private Sector/Self Employed']
uniques of GraduateOrNot is ( 2 ) : 
 ['Yes' 'No']
uniques of AnnualIncome is ( 30 ) : 
 [ 400000 1250000  500000  700000 1150000 1300000 1350000 1450000  800000
 1400000  850000 1500000 1050000  350000 1100000  600000  900000  550000
  300000  750000 1200000 1000000  950000 1700000 1750000  650000  450000
 1650000 1800000 1550000]
uniques of FamilyMembers is ( 8 ) : 
 [6 7 4 3 8 9 5 2]
uniques of ChronicDiseases is ( 2 ) : 
 [1 0]
uniques of FrequentFlyer is ( 2 ) : 
 ['No' 'Yes']
uniques of EverTravelledAbroad is ( 2 ) : 
 ['No' 'Yes']
uniques of TravelInsurance is ( 2 ) : 
 [0 1]


In [23]:
# 動作確認
# df[['Age']]
df['Age'].unique() # ユニークな値一覧(Age内の種類)
# df['Age'].nunique() # ユニークな値の数(種類の数)

array([31, 34, 28, 25, 33, 26, 32, 29, 35, 30, 27])

In [18]:
df.columns

Index(['Age', 'Employment Type', 'GraduateOrNot', 'AnnualIncome',
       'FamilyMembers', 'ChronicDiseases', 'FrequentFlyer',
       'EverTravelledAbroad', 'TravelInsurance'],
      dtype='object')

In [36]:
cat_features = ['Employment Type', 'GraduateOrNot', 'ChronicDiseases', 'FrequentFlyer',
       'EverTravelledAbroad']
num_features = ['Age', 'AnnualIncome', 'FamilyMembers']

In [37]:
df_X = df.drop(columns='TravelInsurance')
df_y = df['TravelInsurance']
X_train, X_test, y_train, y_test = train_test_split(df_X, 
                                                    df_y, 
                                                    test_size=0.2, 
                                                    random_state=17, 
                                                    stratify=df_y)

**stratify** : train, testの間で目的変数(0, 1)の割合が等しくなる

In [38]:
# # 動作確認
# # df_X = df.drop(columns='TravelInsurance') と同じ

# df_X2 = df.drop('TravelInsurance', axis=1)
# df_X2

In [39]:
scaler = StandardScaler()
encoder = OneHotEncoder(drop='first')

**ColumTransformer**
- カラム(列方向の数値型とカテゴリ型)をまとめてエンコードできる

In [40]:
from sklearn.compose import ColumnTransformer

In [44]:
# 各工程のプロセスを作成

# TransformerはScalerやEncoderを指す
# Numericな場合はScaler, StabdardScalerを用いる

ct = ColumnTransformer(
    [
        # (プロセス名, Transformer(Scalerなど), 適応させるカラム名)
        ('num_process', scaler, num_features), # 工程1(各工程のプロセスを作成)
        ('cat_process', encoder, cat_features) # 工程2(各工程のプロセスを作成)
    ],
    remainder = 'passthrough' # or 'drop'
)

remainder = dropにした場合、「num_features, cat_features」以外のカラムを削除

remainder = passthroughにした場合、「num_features, cat_features」以外の(に当てはまらない)カラムはそのまま残される

**これまでとの違い(ColumnTransformerのメリット)**
- これまでは工程1, 工程2を別々で処理してきた(数値データとカテゴリデータの変換)
- このColumnTransformerでは、まとめて一回で処理できる

In [54]:
X_train_transformed = ct.fit_transform(X_train)
X_test_transformed = ct.transform(X_test)
# X_train_transformed[0, :] # 0行目、全部の列
X_train_transformed[0, 0:] # 上と同じ(省略しない場合)

array([ 1.13454572, -0.37974587,  0.76972512,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ])

**数値型で変換された値は「num_features = ['Age', 'AnnualIncome', 'FamilyMembers']」**
- 1.13454572, -0.37974587,  0.76972512

**残りの5つは「cat_features = ['Employment Type', 'GraduateOrNot', 'ChronicDiseases', 'FrequentFlyer',
       'EverTravelledAbroad']」**
- 0. ,1., 0., 0., 0.     

In [57]:
# StandardScalerが正しく働いているかを確認
X_train_transformed[:, 0].std() # 全ての行の0列目(='Age')の標準偏差を確認

1.0

**Next Action**
- モデルの構築

In [60]:
kn_model = KNeighborsClassifier()
sv_model =SVC(random_state=17)

In [62]:
kn_model.fit(X_train_transformed, y_train)
sv_model.fit(X_train_transformed, y_train)

SVC(random_state=17)

In [63]:
print(kn_model.score(X_test_transformed, y_test))
print(sv_model.score(X_test_transformed, y_test))

0.7914572864321608
0.8015075376884422


**GridSearch CV を用いてモデルのパラメータ最適化**

In [70]:
# 辞書型で探索対象を作成
models = [
    {
        'model': kn_model, 
        'params': 
        {
            'n_neighbors': [10, 15, 20, 25], 
            'weights': ['uniform', 'distance']
        }
    },
    {
        'model': sv_model, 
        'params': 
        {
            'C': [0.1, 1, 10, 20], 
            'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
            'gamma': ['auto', 'scale']
        }
    }
        ]

**modelsのリストを使ってGridSearchする**

In [71]:
k = StratifiedKFold(5, shuffle=True, random_state=17)

In [76]:
dfs = []
for model in models:
    grid = GridSearchCV(
        model['model'],
        param_grid=model['params'],
        cv=k # Cross Validation
    )

    grid.fit(X_train_transformed, y_train)
    # fitさせた結果をdfに変換
    df = pd.DataFrame(grid.cv_results_).loc[:, 'params':].sort_values('rank_test_score', ascending=True)[:5]
    # .locで全てのサンプルのうち(:,)'params'以降を取得
    # それらを'rank_test_score'でソート
    # [:5]で上位5つのみ取得
    dfs.append(df)
    print(f"best params of {model['model']} : {grid.best_params_}\n")

best params of KNeighborsClassifier() : {'n_neighbors': 25, 'weights': 'uniform'}

best params of SVC(random_state=17) : {'C': 1, 'gamma': 'scale', 'kernel': 'poly'}



**best paramsを用いて設定**

In [79]:
kn_best_model = KNeighborsClassifier(n_neighbors=25, weights='uniform')
sv_best_model = SVC(C=1, gamma='scale', kernel='poly')

In [80]:
kn_best_model.fit(X_train_transformed, y_train)
sv_best_model.fit(X_train_transformed, y_train)

SVC(C=1, kernel='poly')

In [81]:
print(kn_best_model.score(X_test_transformed, y_test))
print(sv_best_model.score(X_test_transformed, y_test))

0.8015075376884422
0.8040201005025126


**気付き**
- 今回はパラメータ最適化してもあまり精度に変化はなかった
- 今回はデータ分析をせずにモデル構築したが、一旦モデル構築してからデータ分析をしていくのもあり